In [41]:
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm

In [42]:
class MDP:
    def __init__(self):
        self.size = 5
        self.a = (0,1)
        self.b = (0,3)
        self.a_prime = (4,1)
        self.b_prime = (2,3)
        self.actions = [(0,1), (1,0), (0,-1), (-1,0)]
        
    def step(self, state, action):
        if(state == self.a):
            return self.a_prime, 10
        if(state == self.b):
            return self.b_prime, 5
        next_state = tuple(np.array(state) + np.array(action))
        if(next_state[0] < 0 or next_state[1] < 0 or next_state[0] >= self.size or next_state[1] >= self.size):
            return state, -1
        return next_state, 0
        
    def figure32(self):
        m = np.zeros((self.size, self.size), dtype=int)
        c = 0
        for i in range(self.size):
            for j in range(self.size):
                m[i,j] = c
                c+=1
            
        A = np.zeros((self.size**2, self.size**2))
        b = np.zeros(self.size**2)
        
        for i in range(self.size):
            for j in range(self.size):
                state = (i,j)
                for a in range(len(self.actions)):
                    (next_i, next_j), reward = self.step(state, self.actions[a])
#                     print(m[i,j], m[next_i, next_j])
                    A[m[i,j], m[next_i, next_j]] += 0.25*0.9
                    b[m[i,j]] += 0.25*reward
                A[m[i,j], m[i,j]] = A[m[i,j], m[i,j]]-1
        
        X = np.linalg.solve(A,b)
        X = np.round(X, 1)
        for i in range(self.size):
            for j in range(self.size):
                print(X[m[i,j]], end=" ")
            print()
            
class Iterations:
    def __init__(self):
        self.size = 4
        self.terminal = [(0,0), (3,3)]
        self.actions = [(0,1), (1,0), (0,-1), (-1,0)]
        self.policy = np.ones((self.size**2, len(self.actions)))/4
        
    def step(self, state, action):
        if(state in self.terminal):
            return state, 0
        next_state = tuple(np.array(state) + np.array(action))
        if(next_state[0] < 0 or next_state[1] < 0 or next_state[0] >= self.size or next_state[1] >= self.size):
            return state, -1
        return next_state, -1
        
    def policy_iteration(self):
        m = np.zeros((self.size, self.size), dtype=int)
        c = 0
        for i in range(self.size):
            for j in range(self.size):
                m[i,j] = c
                c+=1
                
        value_func = None
        while(True):
            value_func = np.zeros((self.size, self.size))
            #evaluation
            while(True):
                delta = 0
                new_v = np.copy(value_func)
                for i in range(self.size):
                    for j in range(self.size):
                        state = (i,j)
                        value = 0
                        for a in range(len(self.actions)):
                            next_state, reward = self.step(state, self.actions[a])
                            value += self.policy[m[i,j],a]*(reward + value_func[next_state[0], next_state[1]])
                        delta = max(delta, abs(value-value_func[i,j]))
                        value_func[i,j] = value
#                 print(value_func)
#                 print(delta)
#                 value_func = new_v
                if(delta < 1e-4):
                    break
                
#             print(value_func)
            #improvement
            policy_stable = True
            new_policy = []
            for i in range(self.size):
                for j in range(self.size):
                    state = (i,j)
                    action_values = np.zeros(len(self.actions))
                    for a in range(len(self.actions)):
                        next_state, reward = self.step(state, self.actions[a])
                        action_values[a] += (reward + value_func[next_state[0], next_state[1]])
                    
                    old_action = np.argmax(self.policy[m[i,j]])
                    new_action = np.argmax(action_values)
                    
                    if(old_action != new_action):
                        policy_stable = False
                    
                    temp = np.zeros(len(self.actions), dtype=int)
                    temp[new_action] = 1
                    self.policy[m[i,j]] = temp
                    
            if(policy_stable):
                break
                
        print(self.policy)
        print(value_func)
                

In [43]:
mdp = MDP()
mdp.figure32()

-3.3 -8.8 -4.4 -5.3 -1.5 
-1.5 -3.0 -2.3 -1.9 -0.5 
-0.1 -0.7 -0.7 -0.4 0.4 
1.0 0.4 0.4 0.6 1.2 
1.9 1.3 1.2 1.4 2.0 


In [44]:
it = Iterations()
it.policy_iteration()

[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
